# Desafío de Ingeniería de Datos — Análisis de Tweets

## 1. Preparación del entorno

En esta sección, aseguramos que tenemos los archivos necesarios, descargamos los datos desde drive si es necesario, convertimos en parquet para mejor aproveitamento con DuckDB y respondemos las preguntas.

# Contexto y enfoque para resolver el desafío

Este notebook documenta el proceso completo para resolver tres preguntas relacionadas con el análisis de un conjunto de tweets. A continuación se explica la lógica, las decisiones técnicas y las bibliotecas utilizadas para construir una solución eficiente y escalable.

## Elección de herramientas y bibliotecas

- **DuckDB:** Elegí DuckDB como motor de consultas SQL embebido en Python por varias razones:
  - Permite ejecutar consultas SQL directamente sobre archivos Parquet y JSON sin necesidad de cargas complejas.
  - Es muy eficiente para análisis de datos en columnas (columnar), lo que mejora el rendimiento al trabajar con grandes volúmenes de datos.
  - Proporciona un lenguaje declarativo (SQL) para consultas complejas, facilitando la extracción y agregación de información.
  - Comparado con leer archivos directamente con pandas, DuckDB es más rápido y consume menos memoria cuando se manejan grandes datasets.
  - Evita la necesidad de construir procesos ETL o pipelines externos, manteniendo todo en un solo entorno de código.
  - Mayor facilidad de toda la equipa en usar SQL para extraer informaciones.

- **gdown:** Para descargar archivos desde Google Drive de forma automatizada y reproducible.

- **emoji:** Para manusear emojis en contenido de cada tweet y extraer los más utilizados.

## Razonamiento general del enfoque

1. **Preparación de datos:** 
   - El dataset original está en formato JSON, con tweets almacenados línea a línea.
   - Para consultas eficientes, convertimos este JSON a Parquet, un formato columnar optimizado para consultas analíticas, utilizando DuckDB por mejor performance y facilidad.

2. **Consultas analíticas:**
   - Utilizo SQL para filtrar, agrupar y ordenar los datos, logrando respuestas rápidas y claras a las preguntas.
   - Uso expresiones regulares para extraer emojis y menciones dentro del texto de los tweets.
   - Extraigo solo los datos relevantes para cada pregunta, mejorando el rendimiento.

3. **Ventajas de esta solución:**
   - Código limpio y mantenible.
   - Uso de tecnologías que permiten escalabilidad a datasets más grandes.
   - Facilidad para modificar y extender consultas futuras.

---


## 2. Ejecución dos scripts

1. Primero hacemos el download de lo arquivo json, salvo en un drive, usando a lib gdown. Esta funcion armazenará el arquivo json en la pasta data/. Caso el arquivo ya estea en la pasta, el script nos avisará e pulará el processo de download de lo arquivo.

Antes de todo, tenemos que instalar las dependecias:

In [1]:
import os
os.getcwd()

'/home/ebraim/Documentos/projetos/tweet_data/src'

In [2]:
%pip install -r ../requirements.txt

  Using cached memory_profiler-0.61.0-py3-none-any.whl.metadata (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
from download_data import download_file_from_google_drive

In [4]:
%ls 

challenge.ipynb        __init__.py   q1_time.py    q3_memory.py
convert_to_parquet.py  __pycache__/  q2_memory.py  q3_time.py
download_data.py       q1_memory.py  q2_time.py


In [10]:
download_file_from_google_drive()

⬇️ Downloading file...


Downloading...
From (original): https://drive.google.com/uc?id=1B-TywPDU-BBFbMFbrqy2v71FrFxUqsa7
From (redirected): https://drive.google.com/uc?id=1B-TywPDU-BBFbMFbrqy2v71FrFxUqsa7&confirm=t&uuid=e72278b4-1e35-4d9f-bae5-3eca547729e4
To: /home/ebraim/Documentos/projetos/tweet_data/data/farmers-protest-tweets-2021-2-4.json
100%|██████████| 408M/408M [01:54<00:00, 3.56MB/s] 

Elapsed: 119.2029 seconds


2. Después, convertimos el arquivo json para formato parquet, devido a mejor performance para consultas columnares usando DuckDB. Otra véz, caso el arquivo parquet ya estea en la pasta data/, seremos avisados y lo processo de conversion no será ejecutado.

In [5]:
from convert_to_parquet import convert_json_to_parquet

In [6]:
convert_json_to_parquet()

🔄 Converting JSON to Parquet...
✅ Parquet file created: /home/ebraim/Documentos/projetos/tweet_data/data/farmers.parquet
Elapsed: 16.7176 seconds


3. Agora tenemos el arquivo .parquet para hacer consultas optimizadas con duckdb, entoncens podemos responder las preguntas

1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:

In [3]:
import q1_time as q1

In [11]:
q1.execute()

[(datetime.date(2021, 2, 19), 'Preetm91'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 20), 'MangalJ23056160')]
Elapsed: 0.5506 seconds


2. Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:

In [1]:
import q2_time as q2

In [2]:
q2.execute()

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]
Elapsed: 7.2251 seconds


3. El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos. Debe incluir las siguientes funciones:

In [1]:
import q3_time as q3

In [2]:
q3.execute()

[('narendramodi', 2261), ('Kisanektamorcha', 1836), ('RakeshTikaitBKU', 1641), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1015), ('rihanna', 972), ('UNHumanRights', 962), ('meenaharris', 925)]
Elapsed: 1.6351 seconds


In [1]:
import q1_memory as q1_m

In [2]:
q1_m.execute()

Filename: /home/ebraim/Documentos/projetos/tweet_data/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7     71.3 MiB     71.3 MiB           1   @profile
     8                                         def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
     9                                         
    10     71.3 MiB      0.0 MiB           1       CURRENT_FOLDER = Path(__file__).resolve()
    11     71.3 MiB      0.0 MiB           1       PROJECT_ROOT = CURRENT_FOLDER.parent.parent
    12                                         
    13     84.7 MiB     13.4 MiB           1       con = duckdb.connect(database=':memory:')
    14                                         
    15     99.5 MiB     14.8 MiB           3       con.execute(f"""
    16                                                 CREATE TABLE tweets AS 
    17                                                 SELECT 
    18                                                     CAST(

In [1]:
import q2_memory as q2_m

In [2]:
q2_m.execute()

Filename: /home/ebraim/Documentos/projetos/tweet_data/src/q2_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    11     75.0 MiB     75.0 MiB           1   @profile
    12                                         def q2_time(file_path):
    13                                         
    14     75.0 MiB      0.0 MiB           1       CURRENT_FOLDER = Path(__file__).resolve()
    15     75.0 MiB      0.0 MiB           1       PROJECT_ROOT = CURRENT_FOLDER.parent.parent
    16                                         
    17    224.7 MiB    149.8 MiB           1       df = duckdb.query(f"SELECT content FROM read_parquet('{PROJECT_ROOT}/{file_path}')").to_df()
    18                                         
    19    224.7 MiB      0.0 MiB           1       all_emojis = []
    20                                         
    21    231.5 MiB      4.5 MiB      117408       for content in df['content'].dropna():
    22    231.5 MiB      2.3 MiB      117407           all

In [1]:
import q3_memory as q3_m

In [3]:
q3_m.execute()

Filename: /home/ebraim/Documentos/projetos/tweet_data/src/q3_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7     89.8 MiB     89.8 MiB           1   @profile
     8                                         def q3_time(file_path):
     9                                         
    10     89.8 MiB      0.0 MiB           1       CURRENT_FOLDER = Path(__file__).resolve()
    11     89.8 MiB      0.0 MiB           1       PROJECT_ROOT = CURRENT_FOLDER.parent.parent
    12                                             
    13     89.8 MiB      0.0 MiB           3       query = f"""
    14                                                 WITH mentions_extracted AS (
    15                                                     SELECT REGEXP_EXTRACT_ALL(content, '@\\w+') AS mentions
    16     89.8 MiB      0.0 MiB           2               FROM read_parquet('{PROJECT_ROOT}/{file_path}')
    17                                                 ),
    18                 